In [1]:
import pandas as pd
import os


## 通过Ensembl ID作为key，对pVACseq结果和TPM文件合并

In [2]:
TPM_path='/work/longyh/BY/processed/GSE91061_BMS038109Sample.hg19KnownGene.tpm_with_ensembl_offline.csv'
tpm=pd.read_csv(TPM_path, index_col=0)

In [3]:
tpm.shape

(22187, 111)

In [4]:
tpm.columns

Index(['ensembl_gene_id', 'entrez_id_mapped', 'Pt1_Pre_AD101148-6',
       'Pt1_On_AD174047-6', 'Pt10_Pre_E9047565-6', 'Pt10_On_E9047632-6',
       'Pt101_Pre_AD486328-5', 'Pt101_On_AD681975-5', 'Pt103_Pre_AE134058-2',
       'Pt103_On_AE134059-6',
       ...
       'Pt9_Pre_E9021024-6', 'Pt9_On_E9047644-7', 'Pt90_Pre_AD467873-6',
       'Pt92_Pre_AE134060-5', 'Pt92_On_AE148750-5', 'Pt93_On_AE070988-5',
       'Pt94_Pre_AD732850-6', 'Pt94_On_AE373242-6', 'Pt98_Pre_AD733586-8',
       'Pt98_On_AE086717-6'],
      dtype='object', length=111)

In [19]:
tpm.sample(5)

,ensembl_gene_id,entrez_id_mapped,Pt1_Pre_AD101148-6,Pt1_On_AD174047-6,Pt10_Pre_E9047565-6,Pt10_On_E9047632-6,Pt101_Pre_AD486328-5,Pt101_On_AD681975-5,Pt103_Pre_AE134058-2,Pt103_On_AE134059-6,...,Pt9_Pre_E9021024-6,Pt9_On_E9047644-7,Pt90_Pre_AD467873-6,Pt92_Pre_AE134060-5,Pt92_On_AE148750-5,Pt93_On_AE070988-5,Pt94_Pre_AD732850-6,Pt94_On_AE373242-6,Pt98_Pre_AD733586-8,Pt98_On_AE086717-6
9906,ENSG00000215790,9906,5.094934,6.364250,11.521967,10.578114,7.342492,9.705143,1.548983,6.459317,...,15.418141,16.652786,3.227356,8.405293,4.763780,11.455712,4.484263,4.369041,5.040067,5.942939
23077,ENSG00000005810,23077,11.889397,12.073415,29.738098,28.786131,41.603491,64.896197,3.017108,16.329095,...,32.220441,30.113036,7.366086,16.925551,16.385012,14.596292,10.858639,10.959006,15.852199,20.141791
114796,ENSG00000230487,114796,2.858880,5.019185,4.567856,4.664545,6.091442,7.088627,2.602240,4.256077,...,14.715069,13.936002,3.530803,3.767059,4.238990,0.951414,6.140847,4.601766,4.447466,5.277395
80110,ENSG00000142556,80110,8.119681,9.679873,9.658039,10.028355,10.869145,11.203249,2.427923,9.061210,...,21.804147,22.734552,5.600782,2.889161,4.704350,6.982927,7.286731,3.491744,9.418400,10.426586
90075,ENSG00000168661,90075,6.967528,10.156313,11.348416,10.186507,7.137103,8.594582,2.389162,2.962866,...,12.953596,10.531296,3.569863,4.512265,2.389357,3.238548,5.342125,3.146524,7.276627,6.565824


In [5]:
#去除所有带有 _On_ 的列
tpm_filtered=tpm.loc[:,~tpm.columns.str.contains('_On_')]
tpm_filtered.shape

(22187, 53)

In [6]:
# 对于以 'Pt' 开头的列名，只保留第一个 '_' 之前的部分
new_columns = []
for col in tpm_filtered.columns:
    if col.startswith('Pt'):
        new_col = col.split('_')[0]
        new_columns.append(new_col)
    else:
        new_columns.append(col)
tpm_filtered.columns = new_columns
tpm_filtered.shape

(22187, 53)

In [7]:
tpm_filtered.head(5)

,ensembl_gene_id,entrez_id_mapped,Pt1,Pt10,Pt101,Pt103,Pt106,Pt11,Pt17,Pt18,...,Pt8,Pt82,Pt84,Pt85,Pt89,Pt9,Pt90,Pt92,Pt94,Pt98
1,ENSG00000121410,1,8.181307,3.391016,4.331413,12.569035,4.762553,2.024472,2.945181,2.471505,...,6.198876,20.447719,0.708599,10.971456,3.981302,3.437163,5.973345,1.541307,5.064817,4.701648
10,ENSG00000156006,10,0.000000,0.000000,0.000000,0.000000,0.150646,0.000000,0.000000,0.000000,...,0.000000,4.547137,0.171054,0.099993,0.000000,0.606337,0.000000,0.000000,0.055708,0.000000
100,ENSG00000196839,100,27.868108,28.546798,66.517490,31.422172,34.534617,21.138210,10.612183,51.076514,...,22.702733,15.255936,5.930963,12.936927,16.752003,19.749346,29.902344,37.049580,24.663177,9.424623
1000,ENSG00000170558,1000,1.131863,3.372158,3.186713,0.231551,45.952118,1.164447,0.536262,56.516854,...,5.512731,11.045334,3.508698,113.760528,5.094211,98.268472,4.111657,9.877695,2.509534,2.892955
10000,ENSG00000117020,10000,70.242766,132.687164,21.163046,8.201503,84.899175,52.494065,13.442297,51.229632,...,76.697253,61.205530,13.686724,168.680388,31.503918,94.081602,31.270480,58.328645,49.500965,53.206551


In [8]:
tpm_filtered = tpm_filtered.reset_index(drop=True)

In [25]:
tpm_filtered.to_csv('/work/longyh/BY/processed/GSE91061_BMS038109Sample.hg19KnownGene.tpm_with_ensembl_offline.paired.csv', index=False)

查看all_epitope文件

In [9]:
Pt3_path='/work/longyh/BY/processed/naive/Pt3_tumor.MHC_I.all_epitopes.tsv'
pt3=pd.read_csv(Pt3_path, sep='\t')
pt3.columns

/tmp/ipykernel_3201293/3986896029.py:2: DtypeWarning: Columns (15,22) have mixed types. Specify dtype option on import or set low_memory=False.
  pt3=pd.read_csv(Pt3_path, sep='\t')


Index(['Chromosome', 'Start', 'Stop', 'Reference', 'Variant', 'Transcript',
       'Transcript Support Level', 'Transcript Length', 'Canonical',
       'MANE Select', 'Biotype', 'Transcript CDS Flags', 'Ensembl Gene ID',
       'Variant Type', 'Mutation', 'Protein Position', 'Gene Name', 'HGVSc',
       'HGVSp', 'HLA Allele', 'Peptide Length', 'Sub-peptide Position',
       'Mutation Position', 'MT Epitope Seq', 'WT Epitope Seq',
       'Best MT IC50 Score Method', 'Best MT IC50 Score',
       'Corresponding WT IC50 Score', 'Corresponding Fold Change',
       'Best MT Percentile Method', 'Best MT Percentile',
       'Corresponding WT Percentile', 'Tumor DNA Depth', 'Tumor DNA VAF',
       'Tumor RNA Depth', 'Tumor RNA VAF', 'Normal Depth', 'Normal VAF',
       'Gene Expression', 'Transcript Expression', 'Median MT IC50 Score',
       'Median WT IC50 Score', 'Median Fold Change', 'Median MT Percentile',
       'Median WT Percentile', 'NetMHCpan WT IC50 Score',
       'NetMHCpan MT IC50 

In [10]:
pt3.shape

(313140, 59)

In [11]:
pt3.sample(5)

,Chromosome,Start,Stop,Reference,Variant,Transcript,Transcript Support Level,Transcript Length,Canonical,MANE Select,...,Index,Gene of Interest,cterm_7mer_gravy_score,max_7mer_gravy_score,difficult_n_terminal_residue,c_terminal_cysteine,c_terminal_proline,cysteine_count,n_terminal_asparagine,asparagine_proline_bond_count
233965,chr15,42165737,42165738,C,T,ENST00000961229,NaN,872,False,Not Run,...,1015.VPS39.ENST00000961229.missense.587G/S,False,-0.085714,0.428571,False,False,False,0,False,0
119283,chr8,2109438,2109439,G,A,ENST00000953572,NaN,1465,False,Not Run,...,580.MYOM2.ENST00000953572.missense.1030G/R,False,-1.614286,-0.700000,False,False,False,0,False,0
119326,chr6,32522161,32522163,TG,CA,ENST00000714490,NaN,236,False,Not Run,...,533.HLA-DRB5.ENST00000714490.missense.38Q/W,False,-2.014286,-0.928571,False,True,False,1,False,0
97386,chr6,30893138,30893139,C,T,ENST00000937438,NaN,913,False,Not Run,...,516.DDR1.ENST00000937438.missense.391P/S,False,-0.114286,-0.114286,False,False,False,0,False,0
69532,chr3,193492959,193492960,G,T,ENST00000901816,NaN,1155,False,Not Run,...,336.ATP13A4.ENST00000901816.missense.164Q/K,False,-0.028571,0.057143,False,False,False,0,False,0


In [12]:
#将全部为空或NA的列筛查出来
empty_cols = pt3.columns[pt3.isnull().all() | (pt3 == '').all()]
print("全部为空或NA的列有：", empty_cols.tolist())


全部为空或NA的列有： ['Transcript CDS Flags', 'Tumor RNA Depth', 'Tumor RNA VAF', 'Normal Depth', 'Normal VAF', 'Gene Expression', 'Transcript Expression']


In [13]:
#去除这些列
pt3_cleaned = pt3.drop(columns=empty_cols)

In [14]:
pt3_cleaned.columns

Index(['Chromosome', 'Start', 'Stop', 'Reference', 'Variant', 'Transcript',
       'Transcript Support Level', 'Transcript Length', 'Canonical',
       'MANE Select', 'Biotype', 'Ensembl Gene ID', 'Variant Type', 'Mutation',
       'Protein Position', 'Gene Name', 'HGVSc', 'HGVSp', 'HLA Allele',
       'Peptide Length', 'Sub-peptide Position', 'Mutation Position',
       'MT Epitope Seq', 'WT Epitope Seq', 'Best MT IC50 Score Method',
       'Best MT IC50 Score', 'Corresponding WT IC50 Score',
       'Corresponding Fold Change', 'Best MT Percentile Method',
       'Best MT Percentile', 'Corresponding WT Percentile', 'Tumor DNA Depth',
       'Tumor DNA VAF', 'Median MT IC50 Score', 'Median WT IC50 Score',
       'Median Fold Change', 'Median MT Percentile', 'Median WT Percentile',
       'NetMHCpan WT IC50 Score', 'NetMHCpan MT IC50 Score',
       'NetMHCpan WT Percentile', 'NetMHCpan MT Percentile', 'Index',
       'Gene of Interest', 'cterm_7mer_gravy_score', 'max_7mer_gravy_score',


In [15]:
#测试Pt3的'Ensembl Gene ID'有多少可以在tpm_filtered的'tpm_filtered'中找到
matched_genes = pt3_cleaned['Ensembl Gene ID'].isin(tpm_filtered['ensembl_gene_id'])
num_matched = matched_genes.sum()
print(f"在tpm_filtered中找到的基因数量: {num_matched} / {len(pt3_cleaned)}")
print(f"百分比: {num_matched / len(pt3_cleaned) * 100:.2f}%")

在tpm_filtered中找到的基因数量: 310620 / 313140
百分比: 99.20%


In [16]:
pt3["Variant Type"].value_counts()

Variant Type
missense       272052
FS              40800
inframe_del       288
Name: count, dtype: int64

In [21]:
#Mutation Position全部转换为字符串
pt3['Mutation Position'] = pt3['Mutation Position'].astype(str)
pt3['Mutation Position'].value_counts()


Mutation Position
nan                37188
8                  29034
7                  28746
1                  28716
6                  28650
2                  28650
5                  28578
4                  28506
3                  28476
9                  21300
10                 14244
11                  7224
7, 8                 282
7, 8, 9, 10          264
6, 7, 8, 9           264
5, 6, 7, 8           264
8, 9                 258
9, 10                234
10, 11               198
6, 7, 8              186
8, 9, 10, 11         162
9, 10, 11            162
7, 8, 9              162
8, 9, 10             162
6, 7                 108
4, 5, 6, 7           102
6, 7, 8, 9, 11       102
5, 6, 7, 8, 10       102
1, 2                  96
5, 6                  96
3, 4                  96
4, 5                  96
2, 3                  96
1, 3                  72
7, 8, 9, 10, 11       60
6, 7, 8, 9, 10        60
5, 6, 8, 9, 10        24
5, 6, 8               12
6, 7, 9, 10, 11       12
4, 5, 7

In [ ]:
pt3['#'].value_counts()

Biotype
protein_coding    313140
Name: count, dtype: int64

In [23]:
pt3['Transcript Support Level'].value_counts()

Transcript Support Level
1.0    73476
5.0    21132
2.0    20622
3.0     1596
4.0     1368
Name: count, dtype: int64

In [24]:
pt3['MANE Select'].value_counts()

MANE Select
Not Run    313140
Name: count, dtype: int64

In [25]:
pt3['Canonical'].value_counts()

Canonical
False    263820
True      49320
Name: count, dtype: int64

## 用于测试TPM文件中的gene_symbol列能否和pVACseq结果文件中的gene列匹配(已放弃此路线)

In [30]:
TPM_path='/work/longyh/BY/processed/GSE91061_BMS038109Sample.hg19KnownGene.tpm_with_symbol.csv'
# tpm=pd.read_csv(TPM_path,index_col=0)
tpm=pd.read_csv(TPM_path)


In [34]:
tpm.head()

,gene_ID,gene_symbol,Pt1_Pre_AD101148-6,Pt1_On_AD174047-6,Pt10_Pre_E9047565-6,Pt10_On_E9047632-6,Pt101_Pre_AD486328-5,Pt101_On_AD681975-5,Pt103_Pre_AE134058-2,Pt103_On_AE134059-6,...,Pt9_Pre_E9021024-6,Pt9_On_E9047644-7,Pt90_Pre_AD467873-6,Pt92_Pre_AE134060-5,Pt92_On_AE148750-5,Pt93_On_AE070988-5,Pt94_Pre_AD732850-6,Pt94_On_AE373242-6,Pt98_Pre_AD733586-8,Pt98_On_AE086717-6
0,1,A1BG,8.181307,10.642141,3.391016,4.318424,4.331413,3.651002,12.569035,12.539079,...,3.437163,4.621245,5.973345,1.541307,3.406961,3.972505,5.064817,110.887060,4.701648,4.049321
1,10,NAT2,0.000000,0.057901,0.000000,0.115829,0.000000,0.266861,0.000000,0.000000,...,0.606337,0.658621,0.000000,0.000000,0.000000,0.000000,0.055708,29.781233,0.000000,0.000000
2,100,ADA,27.868108,12.344321,28.546798,36.194928,66.517490,78.037661,31.422172,6.781503,...,19.749346,20.485799,29.902344,37.049580,52.096923,5.007066,24.663177,11.180801,9.424623,18.592148
3,1000,CDH2,1.131863,0.119337,3.372158,11.049641,3.186713,7.019172,0.231551,0.051235,...,98.268472,121.799328,4.111657,9.877695,7.786109,0.762186,2.509534,26.071303,2.892955,2.212811
4,10000,AKT3,70.242766,106.608887,132.687164,157.627160,21.163046,25.166711,8.201503,58.732377,...,94.081602,90.423705,31.270480,58.328645,45.222772,76.565628,49.500965,25.485694,53.206551,66.910813


In [33]:
#tpm第一列重命名为gene_ID
tpm.rename(columns={tpm.columns[0]:'gene_ID'}, inplace=True)

In [37]:
#gene_symbol为NA的gene_ID
na_gene_ID=tpm[tpm['gene_symbol'].isna()]['gene_ID'].tolist()

手动查看未匹配的gene ID

In [43]:
# pip install mygene  # 若环境还没有
import mygene

mg = mygene.MyGeneInfo()


In [73]:

entrez_id = "100128573"  # 例：TP53
res = mg.getgene(entrez_id, fields="symbol,name,entrezgene,taxid")
if res is not None:
    print(res["symbol"]+'| '+res["name"]) 
else:
    print(f"No data found for Entrez ID {entrez_id}")
# 若想更稳妥，可用 getgenes 批量：
# res_list = mg.getgenes(["7157", "1956"], fields="symbol,name,entrezgene,taxid")

Input sequence provided is already in string format. No operation performed


No data found for Entrez ID 100128573


In [72]:
from pathlib import Path
import pandas as pd
import re
import mygene

in_path = Path("/work/longyh/BY/processed/na_gene_ID.txt")
out_path = Path("/work/longyh/BY/processed/na_gene_ID_symbol.csv")
unmatched_path = Path("/work/longyh/BY/processed/na_gene_ID_unmatched.txt")  # 可选

# 读取并规范化 ID（去除空行；将形如 "7157.0" 转为 "7157"）
lines = in_path.read_text(encoding="utf-8").splitlines()
ids_raw = [ln.strip() for ln in lines if ln.strip()]

def normalize_id(s: str) -> str:
    s = str(s).strip()
    if re.fullmatch(r"\d+(?:\.0)?", s):
        return s.split(".")[0]
    return s

ids = [normalize_id(x) for x in ids_raw]

mg = mygene.MyGeneInfo()

# 关键：scopes 增加 "retired"，fields 加上 name 兜底，species 保持 human
res = mg.querymany(ids,
                   scopes="entrezgene,retired",
                   fields="symbol,name,taxid",
                   species="human")

# 按原始 query 组装，symbol 缺失时用 name 兜底
by_query = {}
for r in res:
    q = str(r.get("query"))
    if r.get("notfound"):
        by_query[q] = pd.NA
    else:
        sym = r.get("symbol")
        if not isinstance(sym, str) or sym.strip() == "":
            sym = r.get("name")  # 兜底：用 name
        by_query[q] = sym if isinstance(sym, str) else pd.NA

df = pd.DataFrame({"entrez_id": ids})
df["symbol"] = [by_query.get(i, pd.NA) for i in df["entrez_id"]]

df.to_csv(out_path, index=False)
print(f"Saved: {out_path} ({len(df)} rows)")
print(df.head())

# 可选：另存未匹配列表
unmatched = df[df["symbol"].isna()]["entrez_id"].drop_duplicates()
if len(unmatched) > 0:
    unmatched.to_csv(unmatched_path, index=False, header=["entrez_id"])
    print(f"Unmatched IDs saved to: {unmatched_path} ({len(unmatched)} unique)")

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
23 input query terms found no hit:	['100128573', '100130100', '100272216', '100500815', '100500833', '100500892', '100616209', '1006164


Saved: /work/longyh/BY/processed/na_gene_ID_symbol.csv (119 rows)
   entrez_id     symbol
0  100127889     CC2D2B
1  100128338     IQANK1
2  100128573       <NA>
3  100130000  PDE4DIPP6
4  100130093     SNAP47
Unmatched IDs saved to: /work/longyh/BY/processed/na_gene_ID_unmatched.txt (23 unique)


手动查询网站，补充gene symbol

In [74]:
tpm_path = Path("/work/longyh/BY/processed/GSE91061_BMS038109Sample.hg19KnownGene.tpm_with_symbol.csv")
manual_map_path = Path("/work/longyh/BY/processed/na_gene_ID_symbol.csv")
out_tpm_path = Path("/work/longyh/BY/processed/GSE91061_BMS038109Sample.hg19KnownGene.tpm_with_symbol_updated.csv")

In [75]:
tpm=pd.read_csv(tpm_path)
manual_map=pd.read_csv(manual_map_path)

In [76]:
print(manual_map.head())

   entrez_id        symbol
0  100127889        CC2D2B
1  100128338        IQANK1
2  100128573  LOC100128573
3  100130000     PDE4DIPP6
4  100130093        SNAP47


In [77]:
#如果manual_map中symbol不为NA，则更新tpm中的gene_symbol
for index, row in manual_map.iterrows():
    entrez_id = row['entrez_id']
    symbol = row['symbol']
    if pd.notna(symbol):
        tpm.loc[tpm['gene_ID'] == entrez_id, 'gene_symbol'] = symbol

In [79]:
#查看更新后的tpm
tpm['gene_symbol'].isna().sum()

0

In [80]:
#另存
tpm.to_csv(out_tpm_path, index=False)

测试实际匹配情况，以Pt3为例

In [3]:
Pt3_path='/work/longyh/BY/processed/naive/Pt3_tumor.MHC_I.all_epitopes.aggregated.tsv'
pt3=pd.read_csv(Pt3_path,sep='\t')

In [4]:
pt3.columns

Index(['ID', 'Index', 'A*01:01', 'A*23:01', 'B*39:01', 'B*44:03', 'C*04:01',
       'C*12:03', 'Gene', 'AA Change', 'Num Passing Transcripts',
       'Best Peptide', 'Best Transcript', 'MANE Select', 'Canonical', 'TSL',
       'Allele', 'Pos', 'Prob Pos', 'Num Included Peptides',
       'Num Passing Peptides', 'IC50 MT', 'IC50 WT', '%ile MT', '%ile WT',
       'RNA Expr', 'RNA VAF', 'Allele Expr', 'RNA Depth', 'DNA VAF', 'Tier',
       'Evaluation'],
      dtype='object')

In [6]:
print(pt3.shape)
print(tpm.shape)

(184, 32)
(22187, 110)


In [10]:
#pt3的Gene列有多少能在tpm的gene_symbol中找到
matched_genes=pt3['Gene'].isin(tpm['gene_symbol'])
print(f"Number of matched genes: {matched_genes.sum()}")

Number of matched genes: 176


统计所有的naive中基因名匹配情况

In [ ]:
import pandas as pd
from pathlib import Path

tpm_path = Path("/work/longyh/BY/processed/GSE91061_BMS038109Sample.hg19KnownGene.tpm_with_symbol_updated.csv")
naive_dir = Path("/work/longyh/BY/processed/naive")
out_dir = naive_dir / "unmatched_genes"
out_dir.mkdir(exist_ok=True)

tpm = pd.read_csv(tpm_path, index_col=0)
gene_set = set(tpm["gene_symbol"].dropna().astype(str))

summary = []
all_unmatched = []  # 汇总所有文件的未匹配基因（带来源文件）

for f in sorted(naive_dir.glob("*")):
    if not f.is_file():
        continue
    if f.suffix.lower() not in {".tsv", ".csv"}:
        continue

    sep = "\t" if f.suffix.lower() == ".tsv" else ","
    df = pd.read_csv(f, sep=sep)

    if "Gene" not in df.columns:
        continue

    genes = df["Gene"].dropna().astype(str)
    matched = genes.isin(gene_set)
    unmatched_values = genes[~matched].unique()

    # 保存当前文件的未匹配基因到单独文件
    if len(unmatched_values) > 0:
        out_file = out_dir / f"{f.stem}_unmatched_genes.txt"
        pd.Series(unmatched_values).to_csv(out_file, index=False, header=["Gene"])
        all_unmatched.extend([(f.name, g) for g in unmatched_values])

    summary.append({
        "file": f.name,
        "matched_genes": int(matched.sum()),
        "unmatched_genes": int(len(unmatched_values)),
        "total_genes": int(len(genes)),
        "match_ratio": matched.mean(),
        "unique_genes_in_file": int(genes.nunique()),
    })

# 汇总表（匹配统计）
result = pd.DataFrame(summary).sort_values("match_ratio", ascending=False)
print(result)

# 可选：汇总所有未匹配基因及来源文件，便于手动检查
if all_unmatched:
    unmatched_df = pd.DataFrame(all_unmatched, columns=["file", "Gene"])
    unmatched_df.to_csv(out_dir / "all_unmatched_genes_with_source.csv", index=False)
    # 如需去重只留基因名：unmatched_df["Gene"].drop_duplicates().to_csv(out_dir / "all_unmatched_unique_genes.txt", index=False)

In [82]:
result['match_ratio'].value_counts(bins=10).sort_index()

(0.914, 0.923]    1
(0.923, 0.932]    0
(0.932, 0.94]     0
(0.94, 0.949]     1
(0.949, 0.957]    2
(0.957, 0.966]    6
(0.966, 0.974]    9
(0.974, 0.983]    6
(0.983, 0.991]    2
(0.991, 1.0]      6
Name: count, dtype: int64

## 得到RNA seq原始SRA与病人ID的配对

In [3]:
mapping_file='/work/longyh/BY/processed/RNA-seq_SRA_Pt.csv'
mapping=pd.read_csv(mapping_file)

In [4]:
mapping.head()

,accession,sample_accession,sample_alias,sample_title
0,SRR5088813,SRS1844529,GSM2420259,Pt1_Pre_AD101148-6
1,SRR5088814,SRS1844530,GSM2420260,Pt1_On_AD174047-6
2,SRR5088815,SRS1844531,GSM2420261,Pt2_Pre_AD101150-6
3,SRR5088816,SRS1844532,GSM2420262,Pt2_On_AD174046-6
4,SRR5088817,SRS1844533,GSM2420263,Pt23_On_AD486568-8


In [4]:
#mapping中sample_title去掉含有_On_的行
mapping_filtered=mapping[~mapping['sample_title'].str.contains('_On_')]

In [5]:
#mapping中新建一列Patient_ID，从sample_title中提取Pt编号（_前的部分）,使用.loc
mapping_filtered.loc[:, 'Patient_ID'] = mapping_filtered['sample_title'].str.split('_').str[0]

/tmp/ipykernel_3135001/566164444.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapping_filtered.loc[:, 'Patient_ID'] = mapping_filtered['sample_title'].str.split('_').str[0]


In [6]:
mapping_filtered.shape

(51, 5)

In [7]:
mapping_filtered.head()

,accession,sample_accession,sample_alias,sample_title,Patient_ID
0,SRR5088813,SRS1844529,GSM2420259,Pt1_Pre_AD101148-6,Pt1
2,SRR5088815,SRS1844531,GSM2420261,Pt2_Pre_AD101150-6,Pt2
5,SRR5088818,SRS1844534,GSM2420264,Pt23_Pre_AD313075-5,Pt23
6,SRR5088819,SRS1844535,GSM2420265,Pt39_Pre_AD485899-5,Pt39
8,SRR5088821,SRS1844537,GSM2420267,Pt59_Pre_AD823915-5,Pt59


In [8]:
#加载naive队列
naive=pd.read_excel('/work/longyh/BY/processed/merged_WES_meta_cohort_naive.xlsx')
naive.columns

Index(['Patient', 'Cohort', 'Run'], dtype='object')

In [9]:
#mapping_filtered保留Patient_ID存在于naive Patient中的行
mapping_filtered_naive = mapping_filtered[mapping_filtered['Patient_ID'].isin(naive['Patient'])]

In [10]:
mapping_filtered_naive.to_csv('/work/longyh/BY/processed/RNA-seq_naive.csv', index=False)

In [11]:
mapping_filtered_naive['accession'].to_csv('/work/longyh/BY/processed/RNA-seq_naive_accession.txt', index=False, header=False)